In [2]:
#Open Excel File
import openpyxl
import os
import pandas as pd
import numpy as np
import re 

# Load the workbook
workbook = openpyxl.load_workbook('../1st_dataset.xlsx')
worksheet = workbook["Sheet1"]

data = []
headers = []

# Get headers from the first row
for col in range(1, worksheet.max_column + 1):
    headers.append(worksheet.cell(row=1, column=col).value)

# Get data from remaining rows
for row in range(2, worksheet.max_row + 1):
    row_data = []
    for col in range(1, worksheet.max_column + 1):
        row_data.append(worksheet.cell(row=row, column=col).value)
    data.append(row_data)

# Create DataFrame
df = pd.DataFrame(data, columns=headers)

print(headers)

# Display first few rows to verify
print(f"Dataset shape: {df.shape}")
df.head()

['name', 'ingredients', 'ner_ingredient', 'instructions', 'min_age', 'max_age', 'texture', 'prep_time', 'cook_time', 'serving', 'origin', 'recipe_link', 'credibility', 'image_link', 'region', 'difficulty', 'meal_type', 'description', 'dietary_tags', 'choking_hazard', 'tips', 'allergen', 'hypoallergenic', 'nutrition_value', 'ID', 'Energy / Calorie', 'Carbohydrate (g)', 'Protein (g)', 'Fat (g)', 'List of Micros', None, None, None, None, None]
Dataset shape: (1322, 35)


,name,ingredients,ner_ingredient,instructions,min_age,max_age,texture,prep_time,cook_time,serving,...,Energy / Calorie,Carbohydrate (g),Protein (g),Fat (g),List of Micros,None,None,None,None,None
0,Cassava Porridge with Fish Sauce and Lemon (Bu...,"- 60 g cassava, boiled and blended\n- 20 g fis...","['cassava', 'fish', 'chicken', 'coconut oil', ...","Broth:\n1. Use chicken bones, chicken feet, fi...",6,8,None,15 min,45 min,1,...,None,None,None,None,None,None,None,None,None,None
1,Bitterballs (Bitterballen),- 100 g beef mince \n- 30 g potato starch \n- ...,"['beef', 'potato starch', 'milk', 'egg', 'marg...",1. Stir-fry blended spices until fragrant. \n2...,9,11,None,30 minutes,30 minutes,10 servings,...,None,None,None,None,None,None,None,None,None,None
2,Broccoli/Cauliflower Cheese,"- 175g cauliflower/broccoli, cut into pieces\n...","['cauliflower', 'broccoli', 'margarine', 'flou...","1. Steam, boil, or microwave cauliflower/brocc...",6,12,None,~10 min,~20 min,None,...,None,None,None,None,None,None,None,None,None,None
3,Vegetable Fingers,"- 1 carrot, potato, or sweet potato, peeled an...","['carrot', 'potato', 'sweet potato']",1. Steam or microwave vegetables until tender....,6,12,None,~5 min,~10 min,None,...,None,None,None,None,None,None,None,None,None,None
4,Beef Casserole,"- 1 onion, peeled and finely chopped\n- 1½ tab...","['onion', 'vegetable oil', 'beef', 'steak', 'c...",1. Preheat oven to 180°C.\n2. Heat oil in a me...,6,12,None,~10 min,~2.5 hours,None,...,None,None,None,None,None,None,None,None,None,None


In [2]:
# Clean up None columns first
none_columns = [col for col in df.columns if col is None]
if none_columns:
    df = df.drop(columns=none_columns)
    print(f"Dropped {len(none_columns)} None column(s)")


Dropped 5 None column(s)


In [3]:
# Check which records have empty nutrition_value
empty_nutrition = df['nutrition_value'].isna() | (df['nutrition_value'] == '') | (df['nutrition_value'] == 'None')
print(f"Records with empty nutrition_value: {empty_nutrition.sum()} out of {len(df)}")


# Function to extract ingredient information
def extract_ingredient_info(ingredient_text):
    """Extract quantity, measurement, and ingredient name from ingredient text"""
    if pd.isna(ingredient_text) or ingredient_text == '':
        return []
    
    # Split by lines and clean
    lines = [line.strip() for line in ingredient_text.split('\n') if line.strip()]
    
    extracted_ingredients = []
    
    for line in lines:
        # Remove leading dash or bullet points
        line = re.sub(r'^[-•*]\s*', '', line)
        
        # Pattern to match quantity, measurement, and ingredient
        # Examples: "60 g cassava", "2-3 tablespoons of plain yogurt", "1½ tablespoons vegetable oil"
        patterns = [
            # Pattern 1: Range + unit + "of" + ingredient (e.g., "2-3 tablespoons of plain yogurt")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 2: Range + unit + ingredient (e.g., "2-3 tablespoons plain yogurt")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 3: Number + unit + "of" + ingredient (e.g., "30 g of sweet potato")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 4: Number + unit + ingredient (e.g., "60 g cassava", "175g cauliflower")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 5: Fraction + unit + "of" + ingredient (e.g., "1½ tablespoons of oil")
            r'^([0-9]*[½¼¾][0-9]*)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 6: Fraction + unit + ingredient (e.g., "1½ tablespoons oil")
            r'^([0-9]*[½¼¾][0-9]*)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 7: Range + ingredient (no unit) (e.g., "2-3 carrots")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s+(.+)$',
            
            # Pattern 8: Number + ingredient (no unit) (e.g., "1 carrot", "1 onion")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s+(.+)$',
            
            # Pattern 9: Just ingredient (no quantity/unit)
            r'^(.+)$'
        ]
        
        quantity = None
        measurement = None
        ingredient_name = None
        
        for i, pattern in enumerate(patterns):
            match = re.match(pattern, line, re.IGNORECASE)
            if match:
                if i == 0:  # Pattern 1: range + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 1:  # Pattern 2: range + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 2:  # Pattern 3: number + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 3:  # Pattern 4: number + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 4:  # Pattern 5: fraction + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 5:  # Pattern 6: fraction + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 6:  # Pattern 7: range + ingredient (no unit)
                    quantity = match.group(1)
                    measurement = None
                    ingredient_name = match.group(2).strip()
                elif i == 7:  # Pattern 8: number + ingredient (no unit)
                    quantity = match.group(1)
                    measurement = None
                    ingredient_name = match.group(2).strip()
                else:  # Pattern 9: just ingredient
                    quantity = None
                    measurement = None
                    ingredient_name = match.group(1).strip()
                break
        
        # Clean up ingredient name (remove extra descriptions after comma)
        if ingredient_name:
            # Remove descriptions after comma (e.g., "cassava, boiled and blended" -> "cassava")
            ingredient_name = ingredient_name.split(',')[0].strip()
            
            extracted_ingredients.append({
                'original_text': line,
                'quantity': quantity,
                'measurement': measurement,
                'ingredient_name': ingredient_name
            })
    
    return extracted_ingredients



Records with empty nutrition_value: 446 out of 1322


In [4]:
# Test the function with specific examples
print("=== TESTING INGREDIENT EXTRACTION WITH RANGE FORMATS ===")

test_ingredients = [
    "2-3 tablespoons of plain yogurt",
    "1-2 teaspoons of vanilla extract", 
    "30 g of sweet potato",
    "60 g cassava",
    "1½ tablespoons vegetable oil",
    "2-3 carrots",
    "1 onion",
    "plain water"
]

for test_ingredient in test_ingredients:
    print(f"\nTesting: '{test_ingredient}'")
    result = extract_ingredient_info(test_ingredient)
    if result:
        for r in result:
            print(f"  ✅ Quantity: {r['quantity']}, Measurement: {r['measurement']}, Ingredient: {r['ingredient_name']}")
    else:
        print(f"  ❌ No match found")


=== TESTING INGREDIENT EXTRACTION WITH RANGE FORMATS ===

Testing: '2-3 tablespoons of plain yogurt'
  ✅ Quantity: 2-3, Measurement: tablespoons, Ingredient: plain yogurt

Testing: '1-2 teaspoons of vanilla extract'
  ✅ Quantity: 1-2, Measurement: teaspoons, Ingredient: vanilla extract

Testing: '30 g of sweet potato'
  ✅ Quantity: 30, Measurement: g, Ingredient: sweet potato

Testing: '60 g cassava'
  ✅ Quantity: 60, Measurement: g, Ingredient: cassava

Testing: '1½ tablespoons vegetable oil'
  ✅ Quantity: 1½, Measurement: tablespoons, Ingredient: vegetable oil

Testing: '2-3 carrots'
  ✅ Quantity: 2-3, Measurement: None, Ingredient: carrots

Testing: '1 onion'
  ✅ Quantity: 1, Measurement: None, Ingredient: onion

Testing: 'plain water'
  ✅ Quantity: None, Measurement: None, Ingredient: plain water


In [5]:
df_empty_nutrition = df[df['nutrition_value'].isna() | (df['nutrition_value'] == '') | (df['nutrition_value'] == 'None')].copy()

print(f"\nExtracting ingredient information for {len(df_empty_nutrition)} records with empty nutrition_value...")
df_empty_nutrition['extracted_ingredients'] = df_empty_nutrition['ingredients'].apply(extract_ingredient_info)

df_empty_nutrition[['ingredients','extracted_ingredients']]



Extracting ingredient information for 446 records with empty nutrition_value...


,ingredients,extracted_ingredients
0,"- 60 g cassava, boiled and blended\n- 20 g fis...","[{'original_text': '60 g cassava, boiled and b..."
1,- 100 g beef mince \n- 30 g potato starch \n- ...,"[{'original_text': '100 g beef mince', 'quanti..."
2,"- 175g cauliflower/broccoli, cut into pieces\n...","[{'original_text': '175g cauliflower/broccoli,..."
3,"- 1 carrot, potato, or sweet potato, peeled an...","[{'original_text': '1 carrot, potato, or sweet..."
4,"- 1 onion, peeled and finely chopped\n- 1½ tab...","[{'original_text': '1 onion, peeled and finely..."
...,...,...
1317,- 50 g berries (you can use a mix of frozen be...,[{'original_text': '50 g berries (you can use ...
1318,"- 3 small mushrooms, finely chopped\n- ½ cup b...","[{'original_text': '3 small mushrooms, finely ..."
1319,- 1 cup pasta\n- 1 tablespoon margarine\n- 1 t...,"[{'original_text': '1 cup pasta', 'quantity': ..."
1320,- ¼ cup sugar\n- 1 cup milk\n- 1 egg\n- 2 tabl...,"[{'original_text': '¼ cup sugar', 'quantity': ..."


In [6]:
# Display sample extractions
print("\nSample ingredient extractions:")
for idx, row in df_empty_nutrition.head(3).iterrows():
    print(f"\n--- Recipe: {row['name']} ---")
    print(f"Original ingredients:\n{row['ingredients']}")
    print("\nExtracted ingredients:")
    for ing in row['extracted_ingredients']:
        print(f"  - Quantity: {ing['quantity']}, Measurement: {ing['measurement']}, Ingredient: {ing['ingredient_name']}")
        print(f"    Original: {ing['original_text']}")



Sample ingredient extractions:

--- Recipe: Cassava Porridge with Fish Sauce and Lemon (Bubur Singkong Kukuruyuk Saus Jeruk) ---
Original ingredients:
- 60 g cassava, boiled and blended
- 20 g fish meat (milkfish), finely chopped
- 10 g chicken meat
- 5 g coconut oil
- 100 cc chicken broth
- 10 g fresh lime juice
- 20 g spinach, finely chopped

Extracted ingredients:
  - Quantity: 60, Measurement: g, Ingredient: cassava
    Original: 60 g cassava, boiled and blended
  - Quantity: 20, Measurement: g, Ingredient: fish meat (milkfish)
    Original: 20 g fish meat (milkfish), finely chopped
  - Quantity: 10, Measurement: g, Ingredient: chicken meat
    Original: 10 g chicken meat
  - Quantity: 5, Measurement: g, Ingredient: coconut oil
    Original: 5 g coconut oil
  - Quantity: 100, Measurement: cc, Ingredient: chicken broth
    Original: 100 cc chicken broth
  - Quantity: 10, Measurement: g, Ingredient: fresh lime juice
    Original: 10 g fresh lime juice
  - Quantity: 20, Measurement: 

In [7]:
# Create a detailed breakdown DataFrame
ingredient_details = []
for idx, row in df_empty_nutrition.iterrows():
    recipe_name = row['name']
    for ing in row['extracted_ingredients']:
        ingredient_details.append({
            'recipe_id': idx,
            'recipe_name': recipe_name,
            'quantity': ing['quantity'],
            'measurement': ing['measurement'],
            'ingredient_name': ing['ingredient_name'],
            'original_text': ing['original_text']
        })

ingredient_breakdown_df = pd.DataFrame(ingredient_details)
print(f"\nCreated ingredient breakdown with {len(ingredient_breakdown_df)} ingredient entries")
print(f"From {len(df_empty_nutrition)} recipes with empty nutrition values")




Created ingredient breakdown with 2572 ingredient entries
From 446 recipes with empty nutrition values


In [8]:
#print new df
ingredient_breakdown_df.head()


,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60,g,cassava,"60 g cassava, boiled and blended"
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped"
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10,g,chicken meat,10 g chicken meat
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5,g,coconut oil,5 g coconut oil
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100,cc,chicken broth,100 cc chicken broth


In [9]:
# Show summary statistics
print("\nSummary of extracted measurements:")
if len(ingredient_breakdown_df) > 0:
    print(ingredient_breakdown_df['measurement'].value_counts().head(10))
    
    print("\nMost common ingredients:")
    print(ingredient_breakdown_df['ingredient_name'].value_counts().head(10))


Summary of extracted measurements:
measurement
g              556
tsp            181
small          148
tbsp           130
cup             85
ml              79
medium          73
tablespoons     47
teaspoon        45
tablespoon      43
Name: count, dtype: int64

Most common ingredients:
ingredient_name
water            69
vegetable oil    65
onion            48
egg              38
carrot           35
olive oil        34
butter           29
potato           25
eggs             25
sugar            20
Name: count, dtype: int64


In [10]:
# Function to convert measurements to grams
def convert_to_grams(quantity, measurement, ingredient_name):
    """
    Convert quantity and measurement to grams based on common cooking conversions.
    Returns converted quantity in grams and 'g' as measurement.
    """
    if not quantity or not measurement:
        return quantity, measurement
    
    # Clean and parse quantity (handle ranges and fractions)
    def parse_quantity(qty_str):
        if not qty_str:
            return 1.0
        
        # Handle fractions
        fraction_map = {'½': 0.5, '¼': 0.25, '¾': 0.75, '⅓': 0.33, '⅔': 0.67}
        for frac, val in fraction_map.items():
            qty_str = str(qty_str).replace(frac, str(val))
        
        # Handle ranges (take average)
        if '-' in str(qty_str) or '–' in str(qty_str):
            parts = re.split(r'[-–]', str(qty_str))
            if len(parts) == 2:
                try:
                    min_val = float(parts[0].strip())
                    max_val = float(parts[1].strip())
                    return (min_val + max_val) / 2
                except:
                    pass
        
        # Convert to float
        try:
            return float(qty_str)
        except:
            return 1.0
    
    parsed_qty = parse_quantity(quantity)
    measurement_lower = str(measurement).lower()
    
    # Conversion factors to grams
    conversions = {
        # Volume conversions (approximate for common ingredients)
        'tsp': 5,           # 1 tsp ≈ 5g (for most liquids/powders)
        'teaspoon': 5,
        'teaspoons': 5,
        'tbsp': 15,         # 1 tbsp ≈ 15g
        'tablespoon': 15,
        'tablespoons': 15,
        'cup': 240,         # 1 cup ≈ 240g (for liquids)
        'cups': 240,
        'ml': 1,            # 1ml ≈ 1g (for water-based liquids)
        'milliliters': 1,
        'milliliter': 1,
        'l': 1000,          # 1 liter = 1000g
        'liter': 1000,
        'liters': 1000,
        
        # Weight conversions
        'g': 1,             # already in grams
        'gram': 1,
        'grams': 1,
        'kg': 1000,         # 1 kg = 1000g
        'kilogram': 1000,
        'kilograms': 1000,
        'oz': 28.35,        # 1 oz ≈ 28.35g
        'ounce': 28.35,
        'ounces': 28.35,
        'lb': 453.6,        # 1 lb ≈ 453.6g
        'pound': 453.6,
        'pounds': 453.6,
        
        # Piece conversions (rough estimates)
        'small': 50,        # small piece ≈ 50g
        'medium': 100,      # medium piece ≈ 100g
        'large': 150,       # large piece ≈ 150g
        'piece': 75,        # average piece ≈ 75g
        'pieces': 75,
        'clove': 3,         # garlic clove ≈ 3g
        'cloves': 3,
    }
    
    # Convert to grams
    if measurement_lower in conversions:
        converted_qty = parsed_qty * conversions[measurement_lower]
        return round(converted_qty, 1), 'g'
    else:
        # If measurement not found, return original
        return quantity, measurement



In [11]:
# Test the conversion function
print("Testing conversion function:")
test_cases = [
    ('2', 'tbsp', 'vegetable oil'),
    ('1', 'cup', 'flour'),
    ('1', 'tsp', 'salt'),
    ('100', 'ml', 'water'),
    ('1', 'medium', 'onion'),
    ('2-3', 'tsp', 'sugar'),
    ('½', 'cup', 'butter')
]

for qty, measure, ingredient in test_cases:
    new_qty, new_measure = convert_to_grams(qty, measure, ingredient)
    print(f"{qty} {measure} {ingredient} → {new_qty} {new_measure}")


Testing conversion function:
2 tbsp vegetable oil → 30.0 g
1 cup flour → 240.0 g
1 tsp salt → 5.0 g
100 ml water → 100.0 g
1 medium onion → 100.0 g
2-3 tsp sugar → 12.5 g
½ cup butter → 120.0 g


In [19]:
# Apply conversion to all ingredients in the dataframe
print("Converting all measurements to grams...")

# Create new columns for converted values
ingredient_breakdown_df['original_quantity'] = ingredient_breakdown_df['quantity'].copy()
ingredient_breakdown_df['original_measurement'] = ingredient_breakdown_df['measurement'].copy()

# Apply conversion
conversion_results = ingredient_breakdown_df.apply(
    lambda row: convert_to_grams(row['quantity'], row['measurement'], row['ingredient_name']), 
    axis=1
)

# Update the dataframe with converted values
ingredient_breakdown_df['quantity'] = [result[0] for result in conversion_results]
ingredient_breakdown_df['measurement'] = [result[1] for result in conversion_results]

print(f"\nConversion completed for {len(ingredient_breakdown_df)} ingredients")

# Show sample conversions
print("\nSample conversions:")
sample_conversions = ingredient_breakdown_df[ingredient_breakdown_df['original_measurement'].notna()].head(10)
for idx, row in sample_conversions.iterrows():
    print(f"{row['original_quantity']} {row['original_measurement']} {row['ingredient_name']} → {row['quantity']} {row['measurement']}")

# Show new measurement distribution
print("\nNew measurement distribution:")
print(ingredient_breakdown_df['measurement'].value_counts())



Converting all measurements to grams...

Conversion completed for 2572 ingredients

Sample conversions:
60.0 g cassava → 60.0 g
20.0 g fish meat (milkfish) → 20.0 g
10.0 g chicken meat → 10.0 g
5.0 g coconut oil → 5.0 g
100 cc chicken broth → 100 cc
10.0 g fresh lime juice → 10.0 g
20.0 g spinach → 20.0 g
100.0 g beef mince → 100.0 g
30.0 g potato starch → 30.0 g
300.0 g milk → 300.0 g

New measurement distribution:
measurement
g          1547
heaped       16
knob         14
pinch        12
level        11
           ... 
mango         1
square        1
tins          1
English       1
leaves        1
Name: count, Length: 127, dtype: int64


In [13]:
#print new df
ingredient_breakdown_df.head()

,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60,g
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20,g
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10,g
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5,g
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100,cc,chicken broth,100 cc chicken broth,100,cc


In [20]:
# Filter ingredients that have complete quantity and measurement data
print("Filtering ingredients with complete quantity and measurement data...")

# Check current data completeness
print(f"\nTotal ingredients in breakdown: {len(ingredient_breakdown_df)}")
print(f"Ingredients with quantity: {ingredient_breakdown_df['quantity'].notna().sum()}")
print(f"Ingredients with measurement: {ingredient_breakdown_df['measurement'].notna().sum()}")
print(f"Ingredients with both quantity and measurement: {(ingredient_breakdown_df['quantity'].notna() & ingredient_breakdown_df['measurement'].notna()).sum()}")

# Create mask for complete data (both quantity and measurement are not null)
complete_data_mask = (
    ingredient_breakdown_df['quantity'].notna() & 
    ingredient_breakdown_df['measurement'].notna() &
    (ingredient_breakdown_df['quantity'] != '') &
    (ingredient_breakdown_df['measurement'] != '')
)

# Filter dataframe to only include ingredients with complete data
ingredients_complete = ingredient_breakdown_df[complete_data_mask].copy()

print(f"\nIngredients with complete quantity and measurement: {len(ingredients_complete)}")
print(f"Percentage of complete data: {len(ingredients_complete) / len(ingredient_breakdown_df) * 100:.1f}%")

# Show sample of complete ingredients
print("\nSample of ingredients with complete data:")
print(ingredients_complete[['ingredient_name', 'quantity', 'measurement']].head(10))


Filtering ingredients with complete quantity and measurement data...

Total ingredients in breakdown: 2572
Ingredients with quantity: 2061
Ingredients with measurement: 1895
Ingredients with both quantity and measurement: 1895

Ingredients with complete quantity and measurement: 1895
Percentage of complete data: 73.7%

Sample of ingredients with complete data:
        ingredient_name quantity measurement
0               cassava     60.0           g
1  fish meat (milkfish)     20.0           g
2          chicken meat     10.0           g
3           coconut oil      5.0           g
4         chicken broth      100          cc
5      fresh lime juice     10.0           g
6               spinach     20.0           g
7            beef mince    100.0           g
8         potato starch     30.0           g
9                  milk    300.0           g


In [15]:
ingredient_breakdown_df[complete_data_mask]

,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60,g
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20,g
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10,g
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5,g
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100,cc,chicken broth,100 cc chicken broth,100,cc
...,...,...,...,...,...,...,...,...
2567,1321,Apple Crumble,60.0,g,wholemeal self-raising flour,¼ cup wholemeal self-raising flour,¼,cup
2568,1321,Apple Crumble,60.0,g,brown sugar,¼ cup brown sugar,¼,cup
2569,1321,Apple Crumble,60.0,g,coconut,¼ cup coconut,¼,cup
2570,1321,Apple Crumble,60.0,g,rolled oats,¼ cup rolled oats,¼,cup


In [16]:
ingredient_counts = ingredients_complete['ingredient_name'].value_counts()
print(f"Total unique ingredients: {len(ingredient_counts)}")
print("\nMost frequently used ingredients:")
print(ingredient_counts.head(15))

# Create a dataframe with ingredient frequencies
unique_ingredients_with_counts = ingredient_counts.reset_index()
unique_ingredients_with_counts.columns = ['ingredient_name', 'frequency']
print(f"\nCreated dataframe with {len(unique_ingredients_with_counts)} unique ingredients")



Total unique ingredients: 873

Most frequently used ingredients:
ingredient_name
water                   68
vegetable oil           65
onion                   41
olive oil               34
butter                  29
carrot                  23
sugar                   20
garlic                  19
frozen peas             16
potatoes                14
vegetable oil spread    13
flour                   13
lemon juice             12
milk                    12
potato                  12
Name: count, dtype: int64

Created dataframe with 873 unique ingredients


In [17]:

unique_ingredients_with_counts


,ingredient_name,frequency
0,water,68
1,vegetable oil,65
2,onion,41
3,olive oil,34
4,butter,29
...,...,...
868,Natto,1
869,Japanese rice *steamed,1
870,dashi (baby-safe,1
871,udon (preferably thin,1


In [21]:
import requests
from tqdm import tqdm
import requests
import time

api_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
api_key = "KulngHmZ1nJeaPPBrZ8pH3kyJI2Gy1r9Xm121YO9"

def get_nutrition_data(query: str, api_key: str):
    """
    Fetch nutrition data for a specific ingredient from USDA API with prioritized search strategy.
    
    Search Strategy:
    1. Foundation exact match
    2. Foundation first result
    3. Survey (FNDDS) exact match  
    4. Survey (FNDDS) first result
    
    Returns:
    dict: Nutrition information including energy, macronutrients, top 3 micronutrients by value,
          and search method used
    """
    
    base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    
    def search_with_params(data_type):
        """Helper function to search with specific parameters"""
        params = {
            'query': query,
            'api_key': api_key,
            'dataType': [data_type],
            'pageSize': 25,  # Get more results for better matching
            'pageNumber': 1,
            'sortBy': 'dataType.keyword',
            'sortOrder': 'asc'
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=10)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API request failed: {response.status_code}")
                return None
        except Exception as e:
            print(f"❌ Error in API request: {e}")
            return None
    
    def find_exact_match(foods, query_lower):
        """Find exact match by comparing full ingredient name with full food description (case-insensitive)"""
        for food in foods:
            description_lower = food['description'].lower()
            # Check if the query exactly matches the description or if query is a whole word in description
            # Use word boundaries to ensure exact matching
            import re
            
            # Create pattern for exact word matching
            pattern = r'\b' + re.escape(query_lower) + r'\b'
            
            # Check if query is the entire description OR appears as complete word(s)
            if (query_lower == description_lower or 
                re.search(pattern, description_lower)):
                return food
        return None
    
    def extract_nutrition_info(food, search_method):
        """Extract nutrition information from food item"""
        nutrients = food.get('foodNutrients', [])
        
        result = {
            'ingredient_name': query,
            'found_description': food['description'],
            'search_method': search_method,
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'success'
        }
        
        # Energy (Atwater General Factors)
        energy = next((item for item in nutrients if item['nutrientName'] == 'Energy (Atwater General Factors)'), None)
        if energy:
            result['energy_kcal'] = energy['value']
        
        # Carbohydrates
        carbohydrate = next((item for item in nutrients if item['nutrientName'] == 'Carbohydrate, by difference'), None)
        if carbohydrate:
            result['carbohydrate_g'] = carbohydrate['value']
        
        # Fat
        fat = next((item for item in nutrients if item['nutrientName'] == 'Total lipid (fat)'), None)
        if fat:
            result['fat_g'] = fat['value']
        
        # Protein
        protein = next((item for item in nutrients if item['nutrientName'] == 'Protein'), None)
        if protein:
            result['protein_g'] = protein['value']
        
        # Exclude certain nutrients from micronutrients
        exclude_nutrients = [
            "Energy", "Water", "Energy (Atwater General Factors)", "Energy (Atwater Specific Factors)",
            "Nitrogen", "Protein", "Total lipid (fat)", "Ash", "Carbohydrates",
            "Carbohydrate, by difference", "Total dietary fiber (AOAC 2011.25)",
            "High Molecular Weight Dietary Fiber (HMWDF)", "Low Molecular Weight Dietary Fiber (LMWDF)",
            "Sugars, Total", "Total Sugars", "Sucrose", "Glucose", "Fructose", "Lactose", "Maltose"
        ]
        
        # Get micronutrients (vitamins and minerals) - top 3 by value
        filtered_micronutrients = [
            item for item in nutrients 
            if item['nutrientName'] not in exclude_nutrients and item['value'] > 0
        ]
        
        # Sort by value in descending order and take top 3
        sorted_micronutrients = sorted(filtered_micronutrients, key=lambda x: x['value'], reverse=True)
        top_3_micronutrients = sorted_micronutrients[:3]
        
        # Extract only the nutrient names
        micronutrients = [item['nutrientName'] for item in top_3_micronutrients]
        result['micronutrients'] = micronutrients
        
        return result

    # Make the API call with prioritized search strategy
    try:
        query_lower = query.lower()
        
        # Step 1: Search Foundation for exact match
        print(f"🔍 Step 1: Searching Foundation for exact match: '{query}'")
        data = search_with_params("Foundation")
        if data and 'foods' in data and data['foods']:
            exact_match = find_exact_match(data['foods'], query_lower)
            if exact_match:
                print(f"✅ Found exact match in Foundation: {exact_match['description']}")
                return extract_nutrition_info(exact_match, "Foundation_exact")
        
        # Step 2: Search Foundation and take first result
        print(f"🔍 Step 2: Taking first Foundation result: '{query}'")
        if data and 'foods' in data and data['foods']:
            first_result = data['foods'][0]
            print(f"📄 Using first Foundation result: {first_result['description']}")
            return extract_nutrition_info(first_result, "Foundation_first")
        
        # Step 3: Search Survey (FNDDS) for exact match
        print(f"🔍 Step 3: Searching Survey (FNDDS) for exact match: '{query}'")
        data = search_with_params("Survey (FNDDS)")
        if data and 'foods' in data and data['foods']:
            exact_match = find_exact_match(data['foods'], query_lower)
            if exact_match:
                print(f"✅ Found exact match in Survey: {exact_match['description']}")
                return extract_nutrition_info(exact_match, "Survey_exact")
            
            # Step 4: Take first Survey result as fallback
            print(f"🔍 Step 4: Taking first Survey (FNDDS) result: '{query}'")
            first_result = data['foods'][0]
            print(f"📄 Using first Survey result: {first_result['description']}")
            return extract_nutrition_info(first_result, "Survey_first")
        
        # If no results found at all
        print(f"❌ No nutrition data found for '{query}'")
        return {
            'ingredient_name': query,
            'found_description': None,
            'search_method': 'not_found',
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'failed',
            'error': 'No results found in any database'
        }
        
    except Exception as e:
        print(f"❌ Error processing '{query}': {e}")
        return {
            'ingredient_name': query,
            'found_description': None,
            'search_method': 'error',
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'failed',
            'error': str(e)
        }



In [30]:
import time
from tqdm import tqdm

def process_all_ingredients(unique_ingredients_df, api_key, delay=0.1):
    """
    Process all unique ingredients to get their nutrition data.
    
    Args:
        unique_ingredients_df: DataFrame with 'ingredient_name' column
        api_key: USDA API key
        delay: Delay between API calls in seconds (to respect rate limits)
    
    Returns:
        tuple: (nutrition_df, failed_ingredients_list)
    """
    nutrition_results = []
    failed_ingredients = []
    
    print(f"Processing {len(unique_ingredients_df)} unique ingredients...")
    
    # Process each ingredient
    for idx, row in tqdm(unique_ingredients_df.iterrows(), total=len(unique_ingredients_df)):
        ingredient_name = row['ingredient_name']
        # frequency = row['frequency']
        
        try:
            # Get nutrition data
            nutrition_data = get_nutrition_data(ingredient_name, api_key)
            # nutrition_data['frequency'] = frequency
            
            if nutrition_data['status'] == 'success':
                nutrition_results.append(nutrition_data)
                print(f"✅ {ingredient_name}: Found - {nutrition_data['found_description']}")
            else:
                failed_ingredients.append({
                    'ingredient_name': ingredient_name,
                    # 'frequency': frequency,
                    'reason': nutrition_data['status']
                })
                print(f"❌ {ingredient_name}: {nutrition_data['status']}")
            
            # Add delay to respect API rate limits
            time.sleep(delay)
            
        except Exception as e:
            failed_ingredients.append({
                'ingredient_name': ingredient_name,
                # 'frequency': frequency,
                'reason': f'exception: {str(e)}'
            })
            print(f"❌ {ingredient_name}: Exception - {str(e)}")
    
    # Create DataFrames
    nutrition_df = pd.DataFrame(nutrition_results)
    failed_df = pd.DataFrame(failed_ingredients)
    
    return nutrition_df, failed_df

In [25]:
# Process all unique ingredients (excluding water)
print("Starting nutrition data collection for all unique ingredients...")
print(f"Total ingredients to process: {len(unique_ingredients_with_counts)}")

# Exclude 'water' from processing (since water has no significant nutrition and causes matching issues)
ingredients_to_exclude = ['water', 'plain water', 'boiling water', 'cold water', 'warm water']
filtered_ingredients = unique_ingredients_with_counts[
    ~unique_ingredients_with_counts['ingredient_name'].str.lower().isin([x.lower() for x in ingredients_to_exclude])
].copy()


Starting nutrition data collection for all unique ingredients...
Total ingredients to process: 873


In [ ]:

print(f"Processing ALL {len(filtered_ingredients)} ingredients (excluding water)...")
all_nutrition_df, all_failed_df = process_all_ingredients(filtered_ingredients, api_key, delay=0.2)

print(f"\n=== ALL INGREDIENTS PROCESSING COMPLETE ===")
print(f"Successfully processed: {len(all_nutrition_df)} ingredients")
print(f"Failed to process: {len(all_failed_df)} ingredients")
print(f"Success rate: {len(all_nutrition_df) / len(filtered_ingredients) * 100:.1f}%")

print("Note: Processing all ingredients may take a long time due to API rate limits.")

In [ ]:
# Display the nutrition DataFrame in table format
if len(nutrition_df) > 0:
    print("=== NUTRITION DATA SUMMARY ===")
    print(f"Total ingredients with nutrition data: {len(nutrition_df)}")
    
    # Create a clean nutrition DataFrame with the requested columns
    final_nutrition_df = nutrition_df[['ingredient_name', 'found_description', 'search_method',
                                     'energy_kcal', 'carbohydrate_g', 'protein_g', 'fat_g', 
                                     'micronutrients']].copy()
    
    # Create a formatted table for display
    print("\n" + "="*140)
    print("NUTRITION DATA TABLE - WITH PRIORITIZED SEARCH STRATEGY")
    print("="*140)
    
    # Create display dataframe with formatted micronutrients
    display_df = final_nutrition_df.copy()
    display_df['micronutrients_display'] = display_df['micronutrients'].apply(
        lambda x: ', '.join(x[:3]) if x and len(x) > 0 else 'No data'
    )
    
    # Format the table headers
    headers = ['Ingredient', 'Found As', 'Energy\n(kcal)', 'Carbs\n(g)', 'Protein\n(g)', 'Fat\n(g)', 'Top 3 Micronutrients']
    
    # Print header
    print(f"{'Ingredient':<15} {'Found As':<25} {'Search Method':<18} {'Energy':<8} {'Carbs':<8} {'Protein':<8} {'Fat':<8} {'Top 3 Micronutrients':<30}")
    print(f"{'(Input)':<15} {'(USDA Database)':<25} {'(Strategy)':<18} {'(kcal)':<8} {'(g)':<8} {'(g)':<8} {'(g)':<8}")
    print("-" * 140)
    
    # Print data rows
    for idx, row in display_df.iterrows():
        ingredient = str(row['ingredient_name'])[:14]
        found_as = str(row['found_description'])[:24] if row['found_description'] else 'N/A'
        search_method = str(row['search_method'])[:17] if row['search_method'] else 'N/A'
        energy = f"{row['energy_kcal']:.1f}" if pd.notna(row['energy_kcal']) else 'N/A'
        carbs = f"{row['carbohydrate_g']:.1f}" if pd.notna(row['carbohydrate_g']) else 'N/A'
        protein = f"{row['protein_g']:.1f}" if pd.notna(row['protein_g']) else 'N/A'
        fat = f"{row['fat_g']:.1f}" if pd.notna(row['fat_g']) else 'N/A'
        micros = str(row['micronutrients_display'])[:29]
        
        print(f"{ingredient:<15} {found_as:<25} {search_method:<18} {energy:<8} {carbs:<8} {protein:<8} {fat:<8} {micros:<30}")
    
    print("-" * 140)
    
    # Summary statistics table
    print(f"\n{'COLUMN COMPLETENESS SUMMARY':<40}")
    print("-" * 50)
    print(f"{'Column':<20} {'Complete Data':<15} {'Percentage':<15}")
    print("-" * 50)
    total_rows = len(final_nutrition_df)
    
    columns_info = [
        ('Energy (kcal)', final_nutrition_df['energy_kcal'].notna().sum()),
        ('Carbohydrate (g)', final_nutrition_df['carbohydrate_g'].notna().sum()),
        ('Protein (g)', final_nutrition_df['protein_g'].notna().sum()),
        ('Fat (g)', final_nutrition_df['fat_g'].notna().sum()),
        ('Micronutrients', (final_nutrition_df['micronutrients'].str.len() > 0).sum())
    ]
    
    for col_name, complete_count in columns_info:
        percentage = (complete_count / total_rows) * 100
        print(f"{col_name:<20} {complete_count}/{total_rows:<10} {percentage:.1f}%")
    
    print("-" * 50)
    
    # Display the actual DataFrame for further use
    print(f"\nActual DataFrame (for programming use):")
    display(final_nutrition_df)
    
else:
    print("No nutrition data was successfully retrieved.")

In [ ]:
# Display failed ingredients
if len(failed_df) > 0:
    print("=== FAILED INGREDIENTS ===")
    print(f"Total ingredients that could not be processed: {len(failed_df)}")
    
    # Group by failure reason
    failure_reasons = failed_df['reason'].value_counts()
    print(f"\nFailure reasons:")
    for reason, count in failure_reasons.items():
        print(f"  {reason}: {count} ingredients")
    
    print(f"\nList of failed ingredients:")
    for idx, row in failed_df.iterrows():
        print(f"  - {row['ingredient_name']}  - Reason: {row['reason']}")
    
    # Show the failed ingredients DataFrame
    print(f"\nFailed ingredients DataFrame:")
    print(failed_df)
else:
    print("🎉 All ingredients were successfully processed!")

In [ ]:
# Save the results to files
if len(nutrition_df) > 0:
    # Save nutrition data
    nutrition_filename = f"../ingredient_nutrition_data_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    final_nutrition_df.to_excel(nutrition_filename, index=False)
    print(f"✅ Nutrition data saved to: {nutrition_filename}")

if len(failed_df) > 0:
    # Save failed ingredients
    failed_filename = f"../failed_ingredients_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    failed_df.to_excel(failed_filename, index=False)
    print(f"⚠️ Failed ingredients saved to: {failed_filename}")

print(f"\n=== SUMMARY ===")
print(f"✅ Successfully processed: {len(nutrition_df)} ingredients")
print(f"❌ Failed to process: {len(failed_df)} ingredients")
print(f"📊 Success rate: {len(nutrition_df) / (len(nutrition_df) + len(failed_df)) * 100:.1f}%")

# Show the final nutrition DataFrame structure
if len(nutrition_df) > 0:
    print(f"\n=== FINAL NUTRITION DATAFRAME ===")
    print(f"Shape: {final_nutrition_df.shape}")
    print(f"Columns: {list(final_nutrition_df.columns)}")
    final_nutrition_df

In [ ]:
# BATCH PROCESSING: Process ingredients in batches of 150
import math

def process_ingredients_in_batches(ingredients_df, api_key, batch_size=150, delay=0.2, start_batch=1):
    """
    Process ingredients in batches to manage API rate limits and monitor progress.
    
    Args:
        ingredients_df: DataFrame with ingredient data
        api_key: USDA API key
        batch_size: Number of ingredients per batch (default: 150)
        delay: Delay between API calls in seconds
        start_batch: Which batch to start from (useful for resuming)
    
    Returns:
        tuple: (combined_nutrition_df, combined_failed_df, batch_results)
    """
    total_ingredients = len(ingredients_df)
    total_batches = math.ceil(total_ingredients / batch_size)
    
    print(f"📊 BATCH PROCESSING SETUP:")
    print(f"   Total ingredients: {total_ingredients}")
    print(f"   Batch size: {batch_size}")
    print(f"   Total batches: {total_batches}")
    print(f"   Starting from batch: {start_batch}")
    print("=" * 60)
    
    # Store results from all batches
    all_nutrition_results = []
    all_failed_results = []
    batch_summaries = []
    
    for batch_num in range(start_batch, total_batches + 1):
        print(f"\n🔄 PROCESSING BATCH {batch_num}/{total_batches}")
        print("-" * 40)
        
        # Calculate batch indices
        start_idx = (batch_num - 1) * batch_size
        end_idx = min(start_idx + batch_size, total_ingredients)
        
        # Get batch data
        batch_data = ingredients_df.iloc[start_idx:end_idx].copy()
        batch_ingredients = len(batch_data)
        
        print(f"Batch {batch_num}: Processing ingredients {start_idx + 1} to {end_idx} ({batch_ingredients} ingredients)")
        
        try:
            # Process this batch
            batch_nutrition_df, batch_failed_df = process_all_ingredients(
                batch_data, api_key, delay=delay
            )
            
            # Store results
            if len(batch_nutrition_df) > 0:
                all_nutrition_results.append(batch_nutrition_df)
            if len(batch_failed_df) > 0:
                all_failed_results.append(batch_failed_df)
            
            # Calculate batch statistics
            batch_success_rate = len(batch_nutrition_df) / batch_ingredients * 100
            batch_summary = {
                'batch_num': batch_num,
                'ingredients_processed': batch_ingredients,
                'successful': len(batch_nutrition_df),
                'failed': len(batch_failed_df),
                'success_rate': batch_success_rate
            }
            batch_summaries.append(batch_summary)
            
            print(f"\n✅ BATCH {batch_num} COMPLETE:")
            print(f"   Successful: {len(batch_nutrition_df)}/{batch_ingredients} ({batch_success_rate:.1f}%)")
            print(f"   Failed: {len(batch_failed_df)}")
            
            # Save intermediate results (optional)
            if len(batch_nutrition_df) > 0:
                batch_filename = f"../batch_{batch_num}_nutrition_data.xlsx"
                batch_nutrition_df.to_excel(batch_filename, index=False)
                print(f"   💾 Batch results saved: {batch_filename}")
            
        except Exception as e:
            print(f"❌ ERROR in batch {batch_num}: {e}")
            batch_summary = {
                'batch_num': batch_num,
                'ingredients_processed': batch_ingredients,
                'successful': 0,
                'failed': batch_ingredients,
                'success_rate': 0.0,
                'error': str(e)
            }
            batch_summaries.append(batch_summary)
    
    # Combine all results
    print(f"\n🔗 COMBINING ALL BATCH RESULTS...")
    
    # Combine nutrition data
    if all_nutrition_results:
        combined_nutrition_df = pd.concat(all_nutrition_results, ignore_index=True)
    else:
        combined_nutrition_df = pd.DataFrame()
    
    # Combine failed data
    if all_failed_results:
        combined_failed_df = pd.concat(all_failed_results, ignore_index=True)
    else:
        combined_failed_df = pd.DataFrame()
    
    return combined_nutrition_df, combined_failed_df, batch_summaries

# Example: Process ALL ingredients in batches of 150
print("🚀 STARTING BATCH PROCESSING FOR ALL INGREDIENTS")
print(f"Total ingredients to process: {len(filtered_ingredients)} (water excluded)")
print()

# You can modify these parameters:
BATCH_SIZE = 150       
START_BATCH = 1           
API_DELAY = 0.2           

# Uncomment the lines below to start batch processing:
all_nutrition_df, all_failed_df, batch_results = process_ingredients_in_batches(
    filtered_ingredients, 
    api_key, 
    batch_size=BATCH_SIZE, 
    delay=API_DELAY,
    start_batch=START_BATCH
)



🚀 STARTING BATCH PROCESSING FOR ALL INGREDIENTS
Total ingredients to process: 870 (water excluded)

📊 BATCH PROCESSING SETUP:
   Total ingredients: 870
   Batch size: 150
   Total batches: 6
   Starting from batch: 1

🔄 PROCESSING BATCH 1/6
----------------------------------------
Batch 1: Processing ingredients 1 to 150 (150 ingredients)
Processing 150 unique ingredients...


  0%|          | 0/150 [00:00<?, ?it/s]

🔍 Step 1: Searching Foundation for exact match: 'vegetable oil'
🔍 Step 2: Taking first Foundation result: 'vegetable oil'
📄 Using first Foundation result: Oil, canola
✅ vegetable oil: Found - Oil, canola
🔍 Step 2: Taking first Foundation result: 'vegetable oil'
📄 Using first Foundation result: Oil, canola
✅ vegetable oil: Found - Oil, canola


  1%|          | 1/150 [00:01<04:56,  1.99s/it]

🔍 Step 1: Searching Foundation for exact match: 'onion'
✅ Found exact match in Foundation: Green onion, (scallion), bulb and greens, root removed, raw
✅ onion: Found - Green onion, (scallion), bulb and greens, root removed, raw
✅ Found exact match in Foundation: Green onion, (scallion), bulb and greens, root removed, raw
✅ onion: Found - Green onion, (scallion), bulb and greens, root removed, raw


...

🔍 Step 1: Searching Foundation for exact match: 'tofu'
🔍 Step 2: Taking first Foundation result: 'tofu'
🔍 Step 3: Searching Survey (FNDDS) for exact match: 'tofu'
🔍 Step 2: Taking first Foundation result: 'tofu'
🔍 Step 3: Searching Survey (FNDDS) for exact match: 'tofu'
✅ Found exact match in Survey: Soup, miso or tofu
✅ tofu: Found - Soup, miso or tofu
✅ Found exact match in Survey: Soup, miso or tofu
✅ tofu: Found - Soup, miso or tofu


 99%|█████████▊| 148/150 [04:10<00:03,  1.88s/it]

🔍 Step 1: Searching Foundation for exact match: 'coconut oil'
🔍 Step 2: Taking first Foundation result: 'coconut oil'
📄 Using first Foundation result: Oil, coconut
✅ coconut oil: Found - Oil, coconut
🔍 Step 2: Taking first Foundation result: 'coconut oil'
📄 Using first Foundation result: Oil, coconut
✅ coconut oil: Found - Oil, coconut


 99%|█████████▉| 149/150 [04:11<00:01,  1.80s/it]

🔍 Step 1: Searching Foundation for exact match: 'tomato paste'


In [ ]:

BATCH_SIZE = 150       
START_BATCH = 1           
API_DELAY = 0.2           

all_nutrition_df, all_failed_df, batch_results = process_ingredients_in_batches(
    filtered_ingredients, 
    api_key, 
    batch_size=BATCH_SIZE, 
    delay=API_DELAY,
    start_batch=START_BATCH
)


In [28]:
# Display batch processing results and provide resume functionality
def display_batch_summary(batch_results):
    """Display a summary of batch processing results"""
    if not batch_results:
        print("No batch results to display.")
        return
    
    print("\n" + "="*80)
    print("📊 BATCH PROCESSING SUMMARY")
    print("="*80)
    
    # Create summary table
    print(f"{'Batch':<8} {'Ingredients':<12} {'Successful':<12} {'Failed':<8} {'Success Rate':<12} {'Status':<10}")
    print("-" * 80)
    
    total_processed = 0
    total_successful = 0
    total_failed = 0
    
    for result in batch_results:
        batch_num = result['batch_num']
        ingredients = result['ingredients_processed']
        successful = result['successful']
        failed = result['failed']
        success_rate = result['success_rate']
        status = "ERROR" if 'error' in result else "COMPLETE"
        
        print(f"{batch_num:<8} {ingredients:<12} {successful:<12} {failed:<8} {success_rate:<11.1f}% {status:<10}")
        
        total_processed += ingredients
        total_successful += successful
        total_failed += failed
    
    print("-" * 80)
    overall_success_rate = (total_successful / total_processed * 100) if total_processed > 0 else 0
    print(f"{'TOTAL':<8} {total_processed:<12} {total_successful:<12} {total_failed:<8} {overall_success_rate:<11.1f}% {'SUMMARY':<10}")
    print("="*80)
    
    return {
        'total_processed': total_processed,
        'total_successful': total_successful,
        'total_failed': total_failed,
        'overall_success_rate': overall_success_rate
    }

# Uncomment to display results after batch processing:
# if 'batch_results' in locals():
#     summary = display_batch_summary(batch_results)
#     
#     print(f"\n🎉 FINAL RESULTS:")
#     print(f"   Successfully processed: {summary['total_successful']} ingredients")
#     print(f"   Failed to process: {summary['total_failed']} ingredients") 
#     print(f"   Overall success rate: {summary['overall_success_rate']:.1f}%")
#     
#     # Save final combined results
#     if len(all_nutrition_df) > 0:
#         final_filename = f"../final_all_ingredients_nutrition_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#         all_nutrition_df.to_excel(final_filename, index=False)
#         print(f"   💾 Final nutrition data saved: {final_filename}")
#     
#     if len(all_failed_df) > 0:
#         failed_filename = f"../final_failed_ingredients_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#         all_failed_df.to_excel(failed_filename, index=False)
#         print(f"   💾 Failed ingredients saved: {failed_filename}")

print("💡 RESUME FUNCTIONALITY:")
print("   If processing stops, you can resume from any batch by changing START_BATCH parameter")
print("   Example: START_BATCH = 3  # Resume from batch 3")
print("   Individual batch files are saved automatically for backup")

💡 RESUME FUNCTIONALITY:
   If processing stops, you can resume from any batch by changing START_BATCH parameter
   Example: START_BATCH = 3  # Resume from batch 3
   Individual batch files are saved automatically for backup


In [29]:


print("⚠️  TO START BATCH PROCESSING:")
print("   1. Uncomment the lines above")
print("   2. Run this cell")
print("   3. Wait for completion (~25 minutes for all 828 ingredients)")
print()
print(f"📊 PROCESSING PLAN:")
print(f"   • Total ingredients: {len(filtered_ingredients)} (water excluded)")
print(f"   • Batch size: 150 ingredients")
print(f"   • Total batches: {math.ceil(len(filtered_ingredients) / 150)}")
print(f"   • Estimated time: ~{len(filtered_ingredients) * 0.2 / 60:.1f} minutes")
print()
print("💡 FEATURES:")
print("   • Automatic progress tracking")
print("   • Individual batch file saves (for backup)")
print("   • Resume functionality if interrupted")
print("   • Final combined results file")

📊 BATCH PROCESSING SETUP:
   Total ingredients: 870
   Batch size: 150
   Total batches: 6
   Starting from batch: 1

🔄 PROCESSING BATCH 1/6
----------------------------------------
Batch 1: Processing ingredients 1 to 150 (150 ingredients)
❌ ERROR in batch 1: name 'process_all_ingredients' is not defined

🔄 PROCESSING BATCH 2/6
----------------------------------------
Batch 2: Processing ingredients 151 to 300 (150 ingredients)
❌ ERROR in batch 2: name 'process_all_ingredients' is not defined

🔄 PROCESSING BATCH 3/6
----------------------------------------
Batch 3: Processing ingredients 301 to 450 (150 ingredients)
❌ ERROR in batch 3: name 'process_all_ingredients' is not defined

🔄 PROCESSING BATCH 4/6
----------------------------------------
Batch 4: Processing ingredients 451 to 600 (150 ingredients)
❌ ERROR in batch 4: name 'process_all_ingredients' is not defined

🔄 PROCESSING BATCH 5/6
----------------------------------------
Batch 5: Processing ingredients 601 to 750 (150 ingr

In [ ]:
# ===== DISPLAY RESULTS =====
# summary = display_batch_summary(batch_results)
# 
# print(f"\n🎉 FINAL RESULTS:")
# print(f"   Successfully processed: {summary['total_successful']} ingredients")
# print(f"   Failed to process: {summary['total_failed']} ingredients") 
# print(f"   Overall success rate: {summary['overall_success_rate']:.1f}%")
# 
# # Save final combined results
# if len(all_nutrition_df) > 0:
#     final_filename = f"../final_all_ingredients_nutrition_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#     all_nutrition_df.to_excel(final_filename, index=False)
#     print(f"   💾 Final nutrition data saved: {final_filename}")


In [1]:
# Create 6 separate DataFrames for each batch of 150 ingredients
import math

def create_batch_dataframes(unique_ingredients_list, batch_size=150):
    """
    Create 6 separate DataFrames, each containing a batch of ingredients.
    Each DataFrame will be processed separately and can store its own nutrition data.
    
    Args:
        unique_ingredients_list: List of unique ingredients
        batch_size: Number of ingredients per batch (default: 150)
    
    Returns:
        Dictionary of DataFrames (batch_1 through batch_6)
    """
    # Calculate total number of batches needed
    total_batches = math.ceil(len(unique_ingredients_list) / batch_size)
    max_batches = 6  # Limit to 6 batches as requested
    
    print(f"Total ingredients: {len(unique_ingredients_list)}")
    print(f"Batch size: {batch_size}")
    print(f"Total batches needed: {total_batches}")
    print(f"Creating {min(total_batches, max_batches)} DataFrames")
    print("=" * 50)
    
    batch_dataframes = {}
    
    for batch_num in range(1, min(total_batches, max_batches) + 1):
        # Calculate start and end indices for this batch
        start_idx = (batch_num - 1) * batch_size
        end_idx = min(start_idx + batch_size, len(unique_ingredients_list))
        
        # Get ingredients for this batch
        batch_ingredients = unique_ingredients_list[start_idx:end_idx]
        
        # Create DataFrame for this batch with empty nutrition columns
        batch_df = pd.DataFrame({
            'ingredient': batch_ingredients,
            'energy_kcal_per_100g': [None] * len(batch_ingredients),
            'carbs_g_per_100g': [None] * len(batch_ingredients),
            'protein_g_per_100g': [None] * len(batch_ingredients),
            'fat_g_per_100g': [None] * len(batch_ingredients),
            'top_micronutrients': [None] * len(batch_ingredients),
            'search_method': [None] * len(batch_ingredients),
            'batch_number': [batch_num] * len(batch_ingredients)
        })
        
        # Store in dictionary
        batch_dataframes[f'batch_{batch_num}'] = batch_df
        
        print(f"Batch {batch_num}: {len(batch_ingredients)} ingredients (indices {start_idx}-{end_idx-1})")
        print(f"  Sample ingredients: {batch_ingredients[:3]}...")
        print()
    
    return batch_dataframes

# Create the batch DataFrames
print("Creating 6 separate DataFrames for batch processing...")
batch_dfs = create_batch_dataframes(unique_ingredients_list, batch_size=150)

# Display information about each batch
print("\n📊 BATCH DATAFRAMES CREATED:")
print("=" * 60)
for batch_name, df in batch_dfs.items():
    print(f"{batch_name.upper()}:")
    print(f"  - Shape: {df.shape}")
    print(f"  - Ingredient range: {df['ingredient'].iloc[0]} ... {df['ingredient'].iloc[-1]}")
    print(f"  - Batch number: {df['batch_number'].iloc[0]}")
    print()

Creating 6 separate DataFrames for batch processing...


NameError: name 'unique_ingredients_list' is not defined

In [ ]:
# Function to process individual batch DataFrames
def process_batch_dataframe(batch_df, batch_name, api_key, save_results=True):
    """
    Process a single batch DataFrame by fetching nutrition data for all ingredients.
    
    Args:
        batch_df: DataFrame containing ingredients for this batch
        batch_name: Name of the batch (e.g., 'batch_1')
        api_key: USDA API key
        save_results: Whether to save results to Excel file
    
    Returns:
        Tuple of (processed_df, failed_ingredients_list)
    """
    print(f"\n🔄 PROCESSING {batch_name.upper()}")
    print("=" * 50)
    
    processed_df = batch_df.copy()
    failed_ingredients = []
    successful_count = 0
    
    total_ingredients = len(batch_df)
    
    for idx, row in batch_df.iterrows():
        ingredient = row['ingredient']
        
        # Progress indicator
        current_position = idx - batch_df.index[0] + 1
        print(f"Processing {current_position}/{total_ingredients}: {ingredient}")
        
        # Fetch nutrition data
        nutrition_data = get_nutrition_data(ingredient, api_key)
        
        if nutrition_data:
            # Update the DataFrame with nutrition data
            processed_df.at[idx, 'energy_kcal_per_100g'] = nutrition_data.get('energy_kcal_per_100g')
            processed_df.at[idx, 'carbs_g_per_100g'] = nutrition_data.get('carbs_g_per_100g')
            processed_df.at[idx, 'protein_g_per_100g'] = nutrition_data.get('protein_g_per_100g')
            processed_df.at[idx, 'fat_g_per_100g'] = nutrition_data.get('fat_g_per_100g')
            processed_df.at[idx, 'top_micronutrients'] = ', '.join(nutrition_data.get('top_micronutrients', []))
            processed_df.at[idx, 'search_method'] = nutrition_data.get('search_method')
            
            successful_count += 1
            print(f"  ✅ Success - Method: {nutrition_data.get('search_method')}")
        else:
            failed_ingredients.append(ingredient)
            print(f"  ❌ Failed")
        
        # Small delay to avoid overwhelming the API
        time.sleep(0.1)
    
    # Summary
    print(f"\n📊 {batch_name.upper()} SUMMARY:")
    print(f"  - Total ingredients: {total_ingredients}")
    print(f"  - Successful: {successful_count}")
    print(f"  - Failed: {len(failed_ingredients)}")
    print(f"  - Success rate: {successful_count/total_ingredients*100:.1f}%")
    
    # Save results if requested
    if save_results:
        # Save processed DataFrame
        results_filename = f"nutrition_results_{batch_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        processed_df.to_excel(results_filename, index=False)
        print(f"  💾 Results saved to: {results_filename}")
        
        # Save failed ingredients
        if failed_ingredients:
            failed_filename = f"failed_ingredients_{batch_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            failed_df = pd.DataFrame({'failed_ingredient': failed_ingredients, 'batch': batch_name})
            failed_df.to_excel(failed_filename, index=False)
            print(f"  💾 Failed ingredients saved to: {failed_filename}")
    
    return processed_df, failed_ingredients

# Function to process all batches or specific batches
def process_selected_batches(batch_dfs, api_key, batch_numbers=None, save_results=True):
    """
    Process selected batch DataFrames or all batches.
    
    Args:
        batch_dfs: Dictionary of batch DataFrames
        api_key: USDA API key
        batch_numbers: List of batch numbers to process (e.g., [1, 3, 5]) or None for all
        save_results: Whether to save results to files
    
    Returns:
        Dictionary of processed DataFrames and failed ingredients
    """
    processed_results = {}
    all_failed = {}
    
    # Determine which batches to process
    if batch_numbers is None:
        batches_to_process = list(batch_dfs.keys())
    else:
        batches_to_process = [f'batch_{num}' for num in batch_numbers if f'batch_{num}' in batch_dfs]
    
    print(f"🚀 STARTING BATCH PROCESSING")
    print(f"Batches to process: {batches_to_process}")
    print("=" * 60)
    
    for batch_name in batches_to_process:
        if batch_name in batch_dfs:
            try:
                processed_df, failed_list = process_batch_dataframe(
                    batch_dfs[batch_name], 
                    batch_name, 
                    api_key, 
                    save_results
                )
                processed_results[batch_name] = processed_df
                all_failed[batch_name] = failed_list
                
            except Exception as e:
                print(f"❌ Error processing {batch_name}: {str(e)}")
                all_failed[batch_name] = list(batch_dfs[batch_name]['ingredient'])
        
        print("\n" + "="*60)
    
    # Overall summary
    total_ingredients = sum(len(df) for df in batch_dfs.values() if any(batch in batch_dfs for batch in batches_to_process))
    total_successful = sum(len(df[df['search_method'].notna()]) for df in processed_results.values())
    total_failed = sum(len(failed_list) for failed_list in all_failed.values())
    
    print(f"\n🎯 OVERALL PROCESSING SUMMARY:")
    print(f"  - Total ingredients processed: {total_successful + total_failed}")
    print(f"  - Successful: {total_successful}")
    print(f"  - Failed: {total_failed}")
    print(f"  - Overall success rate: {total_successful/(total_successful + total_failed)*100:.1f}%")
    
    return processed_results, all_failed

print("✅ Batch processing functions defined!")
print("\nTo process batches, use:")
print("  - process_selected_batches(batch_dfs, api_key) - Process all batches")
print("  - process_selected_batches(batch_dfs, api_key, [1, 2, 3]) - Process specific batches")
print("  - process_batch_dataframe(batch_dfs['batch_1'], 'batch_1', api_key) - Process single batch")

In [ ]:
# Utility functions for managing batch DataFrames
def display_batch_summary(batch_dfs):
    """Display a summary of all batch DataFrames"""
    print("📋 BATCH DATAFRAMES SUMMARY")
    print("=" * 50)
    
    total_ingredients = 0
    for batch_name, df in batch_dfs.items():
        batch_num = batch_name.split('_')[1]
        processed_count = df['search_method'].notna().sum()
        pending_count = df['search_method'].isna().sum()
        
        print(f"Batch {batch_num}:")
        print(f"  - Total ingredients: {len(df)}")
        print(f"  - Processed: {processed_count}")
        print(f"  - Pending: {pending_count}")
        print(f"  - Sample ingredients: {list(df['ingredient'].head(3))}")
        print()
        
        total_ingredients += len(df)
    
    print(f"Overall: {total_ingredients} ingredients across {len(batch_dfs)} batches")

def get_batch_by_ingredient(batch_dfs, ingredient_name):
    """Find which batch contains a specific ingredient"""
    for batch_name, df in batch_dfs.items():
        if ingredient_name in df['ingredient'].values:
            idx = df[df['ingredient'] == ingredient_name].index[0]
            return batch_name, idx
    return None, None

def combine_batch_results(processed_results):
    """Combine all processed batch DataFrames into a single DataFrame"""
    if not processed_results:
        print("No processed results to combine.")
        return None
    
    combined_df = pd.concat(processed_results.values(), ignore_index=True)
    print(f"Combined {len(processed_results)} batches into single DataFrame with {len(combined_df)} ingredients")
    return combined_df

def save_batch_dataframes(batch_dfs, filename_prefix="batch_dataframes"):
    """Save all batch DataFrames to separate Excel files"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    for batch_name, df in batch_dfs.items():
        filename = f"{filename_prefix}_{batch_name}_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        print(f"Saved {batch_name} to {filename}")

# Display the batch summary
display_batch_summary(batch_dfs)

print("\n" + "="*60)
print("🔧 UTILITY FUNCTIONS AVAILABLE:")
print("  - display_batch_summary(batch_dfs): Show status of all batches")
print("  - get_batch_by_ingredient(batch_dfs, 'apple'): Find which batch contains an ingredient")
print("  - combine_batch_results(processed_results): Combine processed batches into one DataFrame")
print("  - save_batch_dataframes(batch_dfs): Save all batches to Excel files")
print("="*60)

In [ ]:
# 🚀 BATCH PROCESSING EXAMPLES AND INSTRUCTIONS
print("🎯 HOW TO USE THE BATCH PROCESSING SYSTEM")
print("=" * 60)

print("\n1️⃣ PROCESS INDIVIDUAL BATCHES:")
print("   # Process just batch 1")
print("   result_1, failed_1 = process_batch_dataframe(batch_dfs['batch_1'], 'batch_1', api_key)")
print("\n   # Process just batch 2")  
print("   result_2, failed_2 = process_batch_dataframe(batch_dfs['batch_2'], 'batch_2', api_key)")

print("\n2️⃣ PROCESS MULTIPLE SPECIFIC BATCHES:")
print("   # Process batches 1, 3, and 5 only")
print("   results, failed = process_selected_batches(batch_dfs, api_key, [1, 3, 5])")

print("\n3️⃣ PROCESS ALL BATCHES:")
print("   # Process all 6 batches")
print("   all_results, all_failed = process_selected_batches(batch_dfs, api_key)")

print("\n4️⃣ ACCESS INDIVIDUAL DATAFRAMES:")
print("   # Access specific batch")
print("   batch_1_df = batch_dfs['batch_1']")
print("   batch_2_df = batch_dfs['batch_2']")
print("   # ... up to batch_6_df = batch_dfs['batch_6']")

print("\n5️⃣ SAVE AND COMBINE RESULTS:")
print("   # Save individual batches")
print("   save_batch_dataframes(batch_dfs)")
print("   # Combine processed results")
print("   final_df = combine_batch_results(all_results)")

print("\n" + "="*60)
print("📊 CURRENT BATCH STATUS:")

# Show which ingredients are in each batch (first 5 examples)
for i in range(1, min(7, len(batch_dfs) + 1)):
    batch_name = f'batch_{i}'
    if batch_name in batch_dfs:
        df = batch_dfs[batch_name]
        print(f"\nBatch {i} ({len(df)} ingredients):")
        print(f"  First 5: {list(df['ingredient'].head())}")
        if len(df) > 5:
            print(f"  Last 5:  {list(df['ingredient'].tail())}")

print(f"\n📈 READY TO PROCESS {len(unique_ingredients_list)} INGREDIENTS IN {len(batch_dfs)} BATCHES!")
print("💡 TIP: Start with a small batch first to test your API key and settings.")